In [1]:
import torch
import sys
import os
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d

sys.path.append(".")
sys.path.append("..")
sys.path.append(os.path.abspath('D:/Docs/Kursach/stylegan3-editing/utils/'))

from models.stylegan3.model import SG3Generator
from common import make_transform, tensor2im
from editor.Editor import FaceEditor
from models.stylegan3.model import GeneratorType

In [2]:
generator = SG3Generator(checkpoint_path="../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl").decoder

trunc = 0.5

Loading StyleGAN3 generator from path: ../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl
Done!


In [3]:
csv_name = 'iris'
df = pd.read_csv(f"{Path('./datasets/') / csv_name}.csv")
df = df.select_dtypes(exclude=['object'])

In [4]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:])

In [ ]:
num_samples, num_features = df_scaled.shape
df_latents = []
    
for i in range(num_samples):
    f = interp1d(np.linspace(0, 512-1, num=num_features), df_scaled[i], kind='linear')
    vector = f(np.arange(512))
    df_latents.append(vector)

**more_strong_attributes v1**

In [9]:
num_samples, num_features = df_scaled.shape
df_latents=[]
step = 512 // (num_features)
for item in df_scaled:
    vector = np.zeros(512)
    for i in range(num_features):
        vector[i*step:(i+1)*step] = item[i]
    df_latents.append(vector)

**more_strong_attributes v2**

In [10]:
num_samples, num_features = df_scaled.shape
df_latents=[]
for i in range(num_samples):
    f = interp1d(np.linspace(0, 512-1, num=num_features), df_scaled[i], kind='linear')
    vector = f(np.arange(512))
    df_latents.append(vector)

In [151]:
cluster_images = []
for latent in df_latents:
    z = torch.from_numpy(latent).float().to('cuda').unsqueeze(0)
    w = generator.mapping(z, None, truncation_psi=trunc)
    with torch.no_grad():
        img = generator.synthesis(w, noise_mode='const')
        cluster_images.append(tensor2im(img[0]))

start_path = Path(f'./edit_results/{csv_name}')
start_path.mkdir(exist_ok=True, parents=True)
for i, img in enumerate(cluster_images):
    img.save(start_path / f"data_in_image_{i+1}.png")